<a href="https://colab.research.google.com/github/camily-a/SI-not-2021/blob/main/SistemaPerfilSocioeconimico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----- INSTRUÇÕES ----------
- Insira o documento de dados (do tipo .xlsx) na área de arquivos ao lado esquerdo.

- Clique no botão de executar código.

- Insira o nome do arquivo adicionado(com .xlsx).


In [35]:
import pandas as pd #apelido da biblioteca
from datetime import date 
import sys
import plotly.express as px # importa a biblioteca

#name = input('Insira o nome do arquivo de dados (inclui .xlsx) : ')
tabela = pd.read_excel('Resultados.xlsx') #importa dados 

################

tabela = tabela.sort_values(by='Carimbo de data/hora', ascending=True) # organiza tabela por data de envio
tabela = tabela.drop_duplicates(subset=["Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) "], keep="last") #exclui linhas duplicadas mantendo a última enviada

# exclui colunas irrelevantes para a Análise
tabela = tabela.drop("Carimbo de data/hora", axis=1) # axis 1 p/ coluna e 0 para linha
tabela = tabela.drop("Informe os 7 últimos dígitos do seu RA (109048xxxxxxx) ", axis=1)

# mostra todas as linhas da tabela
pd.set_option("max_rows", None)

# coloca os index em sequencia novamente (reseta index)
tabela.reset_index(drop=True, inplace=True)

#################

# trocando valores null/ nan

tabela.fillna("Preferiu não responder", inplace=True)


#############

# convertendo para string e padronizando (strip , lower, upper) 
# pessoas no domicilio, tempo de moradia e empresa que esta contratado

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].astype("string")
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].astype("string")
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].astype("string")

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].str.strip()
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].str.strip()
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].str.strip()

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].str.lower()
tabela['Tempo de moradia neste domicílio.'] = tabela['Tempo de moradia neste domicílio.'].str.lower()
tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].str.upper()


#################

# Descobrindo idade a partir da data de nascimento

data_nasc = (tabela["Qual a sua data de nascimento?"])
days_in_year = 365.25 
agora = pd.Timestamp('today')
 
idade = (agora - data_nasc).dt.days / (days_in_year)
idade = idade.astype(int)

tabela["Qual a sua data de nascimento?"] = idade
tabela.rename(columns={"Qual a sua data de nascimento?": 'Idade'}, inplace = True)

## tempo de moradia (caso = desde que nasci)
tempo = (tabela.index[(tabela['Tempo de moradia neste domicílio.'] == 'desde que nasci')].tolist())

tempo = tabela.iloc[tempo]['Idade'].tolist()

tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace('desde que nasci', tempo[0])

display(tabela["Tempo de moradia neste domicílio."])                                                                                                 
##

############

# agrupando idades


tabela['Idade']=pd.cut(
    tabela['Idade'],
    bins=[-1, 20, 30, 40, sys.maxsize],
    labels=['Até 20 anos', 'Entre 21 e 30 anos', 'Entre 31 e 40 anos', 'Mais de 40 anos']
 )

tabela.rename(columns={"Idade": 'Faixa Etária'}, inplace = True)


########### 

display(tabela)

# padronizando valores iguais , escritos de forma diferente (substituindo)

# pessoas no domicilio

tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['10','dez', 
                                                                                                                                                      '10'], value=10)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['09','Nove', 
                                                                                                                                                      '9'], value=9)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['08','oito', 
                                                                                                                                                       '8'], value=8)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['07','sete', 
                                                                                                                                                       '7'], value=7)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['06','seis', 
                                                                                                                                                       '6'], value=6)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['05','cinco', 
                                                                                                                                                       ' pessoas'], value=5)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['04','quatro', 
                                                                                                                                                       '4'], value=4)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['03','três','tres' 
                                                                                                                                                        '3'], value=3)
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['02','duas', 'dois'
                                                                                                                                                       '2'], value=2)                                                                                                                                                 
tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"] = tabela["Quantas pessoas, incluindo você, moram no seu domicílio?"].replace(regex=['01','uma','um'
                                                                                                                                                          '1'], value=1) 
              

# empresa que esta contratado 

tabela["Qual a empresa que você está contratado agora?"] = tabela["Qual a empresa que você está contratado agora?"].replace(regex=['PREFERIU NÃO RESPONDER','NENHUM',
                                                                                                                              'NENHUMA','NÃO'], value='NÃO TRABALHO')

                                                          
# tempo de moradia

tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['mes','meses','semana','semanas','dias','dia'],
                                                                                                          value = 0.9)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['30', 'trinta'],
                                                                                                          value = 30)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['29' ,'vinte e nove'],
                                                                                                          value = 29)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['28' ,'vinte e oito'],
                                                                                                          value = 28)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['27', 'vinte e sete'],
                                                                                                          value = 27)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['26' ,'vinte e seis'],
                                                                                                          value = 26)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['25' ,'vinte e cinco'],
                                                                                                          value = 25)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['24', 'vinte e quatro'],
                                                                                                          value = 24)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['23', 'vinte e três', 'vinte e tres'],
                                                                                                          value = 23)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['22' ,'vinte e dois'],
                                                                                                          value = 22)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['21' ,'vinte e um'],
                                                                                                          value = 21)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['20', 'vinte'],
                                                                                                          value = 20)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['19' ,'dezenove'],
                                                                                                          value = 19)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['18', 'dezoito'],
                                                                                                          value = 18)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['17', 'dezessete'],
                                                                                                          value = 17)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['16','dezesseis'],
                                                                                                          value = 16)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['15','quinze'],
                                                                                                          value = 15)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['14','quatorze'],
                                                                                                          value = 14)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['13','treze'],
                                                                                                          value = 13)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['12','doze'],
                                                                                                          value = 12)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['11','onze'],
                                                                                                          value = 11)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['10','dez'],
                                                                                                          value = 10)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['09','9','nove'],
                                                                                                          value = 9)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['08','8','oito'],
                                                                                                          value = 8)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['07','7','sete'],
                                                                                                          value = 7)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['06','6','seis'],
                                                                                                          value = 6)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['05','5','cinco'],
                                                                                                          value = 5)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['04','4','quatro'],
                                                                                                          value = 4)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['03','3','tres','três'],
                                                                                                          value = 3)
tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['02','2','dois'],
                                                                                                          value = 2)

tabela["Tempo de moradia neste domicílio."] = tabela["Tempo de moradia neste domicílio."].replace(regex=['01','1', 'um'],
                                                                                                          value = 1)
#############

# agrupando dados (tempo de moradia)

# agrupando tempo de moradia no domicilio

tabela["Tempo de moradia neste domicílio."] = pd.to_numeric(tabela["Tempo de moradia neste domicílio."]) 

tabela["Tempo de moradia neste domicílio."]=pd.cut(
    tabela["Tempo de moradia neste domicílio."],
    bins=[0, 0.9, 5, 10, 20, sys.maxsize],
    labels=['Menos de 1 ano', 'Entre 1 e 5 anos', 'Entre 6 e 10 anos', 'Entre 11 e 20 anos', 'Mais de 20 anos']
 )


# assuntos de jornal


tabela["Se você lê jornal, quais os assuntos que mais lê?"] = tabela["Se você lê jornal, quais os assuntos que mais lê?"].replace(regex=['Notícias locais', 'Notícias nacionais','Lazer', 'arte e cultura',
                                                                                                                                         'Esporte', 'Notícias policiais','Notícias internacionais','Socias'],value = 2)
tabela["Se você lê jornal, quais os assuntos que mais lê?"] = tabela["Se você lê jornal, quais os assuntos que mais lê?"].astype('string')
tabela["Se você lê jornal, quais os assuntos que mais lê?"] = tabela["Se você lê jornal, quais os assuntos que mais lê?"].replace(regex='2',value ='Notícias Específicas')                                                                                                         


##########


display(tabela)

display(tabela.info())


#################

#Criando os gráficos


# loop faz todos os gráficos por coluna da tabela de dados
for coluna in tabela:
  if (coluna != 'Escreva algumas linhas sobre sua história e seus sonhos de vida'):
    qtd = tabela[coluna].value_counts()  #conta os dados da coluna
    if ( coluna == 'Qual a empresa que você está contratado agora?') :
       grafico = px.bar(y = qtd.index,
                        x= qtd.values, orientation='h', width=1000, height=1000,
              title=coluna, color_discrete_sequence=px.colors.sequential.RdBu)
       grafico.update_traces(textposition='inside',texttemplate='%{x}')
       labels={'y': 'Empresa',
               'x': 'Número de Alunos'}
       grafico.update_traces(hovertemplate = "%{label} <br>N° de Alunos: %{value}")
    else:
      # faz o gráfico de pizza usando os dados contados
      grafico = px.pie(names = qtd.index,
                        values = qtd.values,
                title=coluna, color_discrete_sequence=px.colors.sequential.RdBu) 
      # algumas formatações do gráfico
      grafico.update_traces(hovertemplate = "%{label} <br>N° de Alunos: %{value}", textfont_size=20,
                    title_font_color='#000000', title_font_size=20)
    # mostra gráfico
    display(grafico)
  else :
    print ('''\n Para vizualização das respostas da questão: ESCREVA ALGUMAS LINHAS SOBRE SUA HISTÓRIA E SEUS SONHOS DE VIDA ,
em formato de nuvem de palavras ,copie o texto abaixo e insira na página https://www.wordclouds.com/   \n 
---- TEXTO ----- :''')
    display('''Em 2020 comecei a estudar programação sozinho em casa, em 2021 entrei pra FATEC e conquistei uma vaga de estagio.Espero que futuramente consiga um bom salario
Estudei a vida inteira em escola pública. Quando terminei o ensino médio não sabia qual faculdade iria cursar e foi quando me sugeriram dar uma olhada nos cursos que a FATEC oferece. Por fim eu escolhi o curso de ADS que foi o que eu mais me identifiquei. Pretendo me formar na área e melhorar meus conhecimentos para poder auxiliar ainda mais no meu trabalho.
ter uma vida tranquila 
Sou um amante de TI e sonho em ser bem sucedido na minha área 
Conhecer o Brasil 
Não tenho muita coisa pra contar, mas espero me formar na faculdade e atuar na área, assim, obtendo sucesso profissional e sendo bem sucedida. 
Gosto muito da área de informática, sou uma pessoa muito curiosa e dedicada, pretendo seguir nessa área
Mudar de pais
Meu nome é Guilherme Vinaud, tenho 30 anos e me interessei pela área de TI depois que fiz um curso na Etec. Não trabalho na área ainda mas pretendo me formar e conquistar uma vaga neste seguimento de mercado .
Entrei na Fatec com o objetivo de me capacitar profissionalmente e espero conquistar uma vaga no mercado de trabalho como programador. 
Desejo me formar na área sendo muito bem capacitado e conseguir um bom emprego para realizar meus sonhos mundo a fora.
Sempre me identifiquei na área da tecnologia e tive a oportunidade de estudar na área e espero conquistar um emprego bom 
Sempre gostei muito da área, mas nunca me aprofundei muito. Com os conhecimentos adquiridos no curso pretendo seguir a área de segurança da informação.
Ficar bilionário
Espero me profissionalizar na área e obter novos conhecimentos
Pretendo me tornar um ótimo profissional da area e melhorar minha renda.
Nada a declarar.
Nasci em 2003, o meu primeiro contato com a tecnologia vou atravez de video-game e ficava encantado e pensando como que criaram isso foi por causa disso eu comecei a me interessar em tecnologia, com isso fiz um curso de criação de jogos que me introduziu aos script foi dai que comecei a olhar para cursos que ensinam script como analise e desenvolvimento de sistema e concluindo poderei criar jogos e trabalhar com scripts
Nascida em Franca,casada,tenho como objetivo principal a formação e conquista do mercado de trabalho... 
Sempre gostei da area de tecnologia, desmontava meus carrinhos de controle remoto e quando era criança para ver por dentro, estudei na Escola Julio Cardoso no ensino medio e também fiz o técnico em eletronica e me eltrotécnica. Cursei engenharia de automação e controle na Unip-RP porém sempre trabalhei na área de telecomunicações e agora pretendo entrar na área de desenvolviemento de sistemas.
Sou uma pessoa simples. Como qualquer outro brasileiro que busca um lugar ao sol. Tenho sonho de conquistar minha casa própria. 
Ser um bom analista e fazer a diferença no mundo da tecnologia
Obter uma estabilidade profissional, com um salário razoável, que vai me proporcionar a viajar e conhecer várias cidades e países . 
Tenho 18 anos, estudei minha vida inteira em escola pública .Sempre fui uma pessoa quieta e reservada. Meu objetivo é terminar a faculdade e conseguir uma vaga de emprego no ramo da tecnologia, e então continuar aprimorar meus conhecimentos profissionais. 
Amo tecnologia desde os 9 anos de idade, pertenço a uma família pobre mas que sempre trabalhou muito. Sonho em encontrar uma maneira de mudar o mundo, ser reconhecido, e de principalmente ajudar as pessoas vejo na tecnologia e na programação maneiras de realizar estes sonhos.
Tenho 19 anos, concluo em 2021 técnico em Desenvolvimento de Sistemas e técnico em Mecatrônica. Almejo obter uma vaga em uma grande empresa de automação industrial e posteriormente abrir meu próprio negócio
Olá sou Victor Hugo Pasqual , meu sonho abrir meu proprio negocio na area de desenvolvimento, e talvez fora do pais 
Sou uma pessoa de famílias humilde, o primeiro a ingressar em uma faculdade, gosto da área e sei que é a minha vocação. Tenho o sonho de me formar e ter a minha própria empresa e ajudar todos os meus familiares
Trabalho desde novo, e meu sonho sempre foi estudar na area de programação, e espero conseguir um bom emprego. E poder viajar bastante. 
Sou formada em química e não tive uma boa oportunidades na área em seguida surgiu a oportunidade de trabalhar com informática e me apaixonei pela área. O meu objetivo é consegui uma vaga em uma grande empresa, organizada e com um plano de carreira.
Desde novo utilizo computador, criei um servidor de MuOnline no qual já está online a + de 4 anos, e amo essa aréa
Já estou em franca a 18 anos, sou bem familiar, tenho como objetivo entrar para PF (Polícia federal). 
Quero me formar na faculdade e atuar na área sendo um profissional bem sucedido
Nasci e moro em Franca, tenho 18 anos. Moro com meus pais e dois irmãos, sou o caçula. Conheci e comecei esse curso por causa do meu irmão mais velho que me motivou e me ensinou algumas coisas de python, eu gostei, e comecei a estudar e entrei na Fatec (vimos a inscrição na página da Fatec no Facebook). Nas horas vagas eu toco kalimba, ouço música, estudo não só linguagens de programação mas também biologia e as vezes hebraico. Alguns dos meus sonhos são: conseguir um emprego bom nessa área de tecnologia e obter mais conhecimento, viajar para outros lugares e/ou países, fazer também faculdade de biologia (só por gostar mesmo).
Tenho 18 anos, comecei a gostar de tecnologia ano passado, foi ai que tentei vaga na FATEC. Ainda não trabalho na área, mais pretendo ingressar logo. Meu sonho é trabalhar nela. 
Meu nome é Luiz Felipe Silva, tenho 25 anos, sou natural de Franca-SP. Sou apaixonado por basquete, vídeo games e tecnologia. Já atuei na área, porém no momento trabalho com atendimento ao cliente.  
Minha historia começa no Estado do Amazonas e até o momento está no Sudeste, adoro novos lugares, novos conhecimentos meu sonho é poder fazer algum trabalho futuro com informática que faça a diferença na vida das pessoas.
Quero ajudar a Web ser mais acessível para deficientes visuais
Sou uma pessoa de fácil convívio, gosto de me comunicar, de aprender coisas novas e tenho muita força de vontade e perseverança. Com este curso, pretendo entrar na área  e aprender o máximo possível para que consiga ser um bom analista e me realizar profissionalmente.
Meu nome é Rafael, programo desde meus 16 anos, sempre soube que entraria na carreira da computação. Meu sonho é poder ser autônomo e não tiver horário fixo de trabalho
Não tem muito o que falar, sou um estudante que por conta da pandemia deixou de frequentar a faculdade. Estou tentando voltar a faculdade para me formar!''')



0                1 ano
1               2 anos
2              9 meses
3              18 anos
4                   21
5              24 anos
6               14anos
7                   15
8                    2
9              17 anos
10                  17
11                  18
12                   5
13               1 ano
14             17 anos
15              4 anos
16            +10 anos
17             16 anos
18             23 anos
19             8 meses
20              7 anos
21             20 anos
22             9 meses
23            1 semana
24             7 meses
25              7 anos
26             18 anos
27             15 anos
28      dezessete anos
29        mais 20 anos
30               1 ano
31             18 anos
32              2 anos
33             18 anos
34             18 anos
35             6 meses
36                  4+
37                  18
38            18 anos.
39             18 anos
40             12 anos
41             10 anos
42             29 anos
43         

,Qual o seu curso?,Qual o período que cursa?,Qual o estado do Brasil que você nasceu?,Qual sua cidade de residência?,Qual o seu gênero?,Faixa Etária,Qual é o seu estado civil?,"Você é portador de alguma deficiência? (Pode selecionar mais de uma, se for o caso)",Quantos filhos você tem?,Com quem você mora atualmente?,"Quantas pessoas, incluindo você, moram no seu domicílio?",Qual a situação do domicílio onde mora?,Tempo de moradia neste domicílio.,Qual a faixa de renda mensal da sua família (em Salários Mínimos)?,Quantos de cada um dos itens abaixo há em seu domicílio? [Televisor:],Quantos de cada um dos itens abaixo há em seu domicílio? [Video cassete e(ou) DVD:],Quantos de cada um dos itens abaixo há em seu domicílio? [Rádio:],Quantos de cada um dos itens abaixo há em seu domicílio? [Automóvel: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Motocicleta:],Quantos de cada um dos itens abaixo há em seu domicílio? [Máquina de lavar roupa e(ou) Tanquinho:],Quantos de cada um dos itens abaixo há em seu domicílio? [Geladeira: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Celular comum: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Smartphone:],Quantos de cada um dos itens abaixo há em seu domicílio? [Microcomputador de mesa:],Quantos de cada um dos itens abaixo há em seu domicílio? [Notebook: ],No seu domicílio você tem: [Telefone fixo:],No seu domicílio você tem: [Internet:],No seu domicílio você tem: [TV por assinatura:],No seu domicílio você tem: [Empregada mensalista:],Você trabalha?,Qual o seu vínculo com o emprego?,Qual a área do seu trabalho?,Qual seu horário de trabalho?,Qual a empresa que você está contratado agora?,Você tem plano de saúde privado?,Qual o grau de escolaridade do seu pai?,Qual o grau de escolaridade da sua mãe?,Na sua vida escolar você estudou...,Com que frequência você utiliza microcomputadores?,Onde você utiliza microcomputadores? [Em casa:],...,Onde você utiliza microcomputadores? [Na escola:],Onde você utiliza microcomputadores? [Em outros lugares:],Com qual finalidade você utiliza microcomputadores? [Para trabalhos profissionais: ],Com qual finalidade você utiliza microcomputadores? [Para trabalhos escolares:],"Com qual finalidade você utiliza microcomputadores? [Para entretenimento (músicas, vídeos, redes sociais, etc): ]",Com qual finalidade você utiliza microcomputadores? [Para comunicação por e-mail: ],Com qual finalidade você utiliza microcomputadores? [Para operações bancárias: ],Com qual finalidade você utiliza microcomputadores? [Para compras eletrônicas: ],Como você classifica seu conhecimento em informática?,Qual o seu conhecimento em relação aos aplicativos a seguir? [Windows:],Qual o seu conhecimento em relação aos aplicativos a seguir? [Linux:],"Qual o seu conhecimento em relação aos aplicativos a seguir? [Editores de textos (Word, Writer, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Planilhas eletrônicas (Excel, Calc, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Apresentadores (Powerpoint, Impress, Prezzi, etc.):]",Qual o seu conhecimento em relação aos aplicativos a seguir? [Sistemas de Gestão Empresarial:],Inglês,Espanhol,Outro idiomas,Considere a busca por informação nos seguintes meios de comunicação: [TV],Considere a busca por informação nos seguintes meios de comunicação: [Internet],Considere a busca por informação nos seguintes meios de comunicação: [Revistas],Considere a busca por informação nos seguintes meios de comunicação: [Jornais],Considere a busca por informação nos seguintes meios de comunicação: [Rádio],Considere a busca por informação nos seguintes meios de comunicação: [Redes sociais],Considere a busca por informação nos seguintes meios de comunicação: [Conversas com Amigos],"Se você lê jornal, qual a frequência?","Se você lê jornal, quais os assuntos que mais lê?","Não considerando os livros escolares, quantos livros você lê por ano (em média)?","Se você lê livros literários, 

,Qual o seu curso?,Qual o período que cursa?,Qual o estado do Brasil que você nasceu?,Qual sua cidade de residência?,Qual o seu gênero?,Faixa Etária,Qual é o seu estado civil?,"Você é portador de alguma deficiência? (Pode selecionar mais de uma, se for o caso)",Quantos filhos você tem?,Com quem você mora atualmente?,"Quantas pessoas, incluindo você, moram no seu domicílio?",Qual a situação do domicílio onde mora?,Tempo de moradia neste domicílio.,Qual a faixa de renda mensal da sua família (em Salários Mínimos)?,Quantos de cada um dos itens abaixo há em seu domicílio? [Televisor:],Quantos de cada um dos itens abaixo há em seu domicílio? [Video cassete e(ou) DVD:],Quantos de cada um dos itens abaixo há em seu domicílio? [Rádio:],Quantos de cada um dos itens abaixo há em seu domicílio? [Automóvel: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Motocicleta:],Quantos de cada um dos itens abaixo há em seu domicílio? [Máquina de lavar roupa e(ou) Tanquinho:],Quantos de cada um dos itens abaixo há em seu domicílio? [Geladeira: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Celular comum: ],Quantos de cada um dos itens abaixo há em seu domicílio? [Smartphone:],Quantos de cada um dos itens abaixo há em seu domicílio? [Microcomputador de mesa:],Quantos de cada um dos itens abaixo há em seu domicílio? [Notebook: ],No seu domicílio você tem: [Telefone fixo:],No seu domicílio você tem: [Internet:],No seu domicílio você tem: [TV por assinatura:],No seu domicílio você tem: [Empregada mensalista:],Você trabalha?,Qual o seu vínculo com o emprego?,Qual a área do seu trabalho?,Qual seu horário de trabalho?,Qual a empresa que você está contratado agora?,Você tem plano de saúde privado?,Qual o grau de escolaridade do seu pai?,Qual o grau de escolaridade da sua mãe?,Na sua vida escolar você estudou...,Com que frequência você utiliza microcomputadores?,Onde você utiliza microcomputadores? [Em casa:],...,Onde você utiliza microcomputadores? [Na escola:],Onde você utiliza microcomputadores? [Em outros lugares:],Com qual finalidade você utiliza microcomputadores? [Para trabalhos profissionais: ],Com qual finalidade você utiliza microcomputadores? [Para trabalhos escolares:],"Com qual finalidade você utiliza microcomputadores? [Para entretenimento (músicas, vídeos, redes sociais, etc): ]",Com qual finalidade você utiliza microcomputadores? [Para comunicação por e-mail: ],Com qual finalidade você utiliza microcomputadores? [Para operações bancárias: ],Com qual finalidade você utiliza microcomputadores? [Para compras eletrônicas: ],Como você classifica seu conhecimento em informática?,Qual o seu conhecimento em relação aos aplicativos a seguir? [Windows:],Qual o seu conhecimento em relação aos aplicativos a seguir? [Linux:],"Qual o seu conhecimento em relação aos aplicativos a seguir? [Editores de textos (Word, Writer, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Planilhas eletrônicas (Excel, Calc, etc.):]","Qual o seu conhecimento em relação aos aplicativos a seguir? [Apresentadores (Powerpoint, Impress, Prezzi, etc.):]",Qual o seu conhecimento em relação aos aplicativos a seguir? [Sistemas de Gestão Empresarial:],Inglês,Espanhol,Outro idiomas,Considere a busca por informação nos seguintes meios de comunicação: [TV],Considere a busca por informação nos seguintes meios de comunicação: [Internet],Considere a busca por informação nos seguintes meios de comunicação: [Revistas],Considere a busca por informação nos seguintes meios de comunicação: [Jornais],Considere a busca por informação nos seguintes meios de comunicação: [Rádio],Considere a busca por informação nos seguintes meios de comunicação: [Redes sociais],Considere a busca por informação nos seguintes meios de comunicação: [Conversas com Amigos],"Se você lê jornal, qual a frequência?","Se você lê jornal, quais os assuntos que mais lê?","Não considerando os livros escolares, quantos livros você lê por ano (em média)?","Se você lê livros literários, 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 81 columns):
 #   Column                                                                                                               Non-Null Count  Dtype   
---  ------                                                                                                               --------------  -----   
 0   Qual o seu curso?                                                                                                    65 non-null     object  
 1   Qual o período que cursa?                                                                                            65 non-null     object  
 2   Qual o estado do Brasil que você nasceu?                                                                             65 non-null     object  
 3    Qual sua cidade de residência?                                                                                      65 non-null     object  
 4   Qual o seu

None


 Para vizualização das respostas da questão: ESCREVA ALGUMAS LINHAS SOBRE SUA HISTÓRIA E SEUS SONHOS DE VIDA ,
em formato de nuvem de palavras ,copie o texto abaixo e insira na página https://www.wordclouds.com/   
 
---- TEXTO ----- :


'Em 2020 comecei a estudar programação sozinho em casa, em 2021 entrei pra FATEC e conquistei uma vaga de estagio.Espero que futuramente consiga um bom salario\nEstudei a vida inteira em escola pública. Quando terminei o ensino médio não sabia qual faculdade iria cursar e foi quando me sugeriram dar uma olhada nos cursos que a FATEC oferece. Por fim eu escolhi o curso de ADS que foi o que eu mais me identifiquei. Pretendo me formar na área e melhorar meus conhecimentos para poder auxiliar ainda mais no meu trabalho.\nter uma vida tranquila \nSou um amante de TI e sonho em ser bem sucedido na minha área \nConhecer o Brasil \nNão tenho muita coisa pra contar, mas espero me formar na faculdade e atuar na área, assim, obtendo sucesso profissional e sendo bem sucedida. \nGosto muito da área de informática, sou uma pessoa muito curiosa e dedicada, pretendo seguir nessa área\nMudar de pais\nMeu nome é Guilherme Vinaud, tenho 30 anos e me interessei pela área de TI depois que fiz um curso na E